In [1]:
# !pip install mxnet
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece
# !pip install transformers
# !pip install torch

In [2]:
# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [61]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import mxnet.gluon.data

# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [62]:
## GPU
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /home/analysis-2/tipa_analysis/.cache/kobert_v1.zip
using cached model. /home/analysis-2/tipa_analysis/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [63]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /home/analysis-2/tipa_analysis/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [64]:
import os
import pyarrow.parquet as pq

# dataset_train = nlp.data.TSVDataset(".cache/ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset = pq.read_table('tipa_text_tokens_20220328.parquet').to_pandas()
dataset['label'] = 0
dataset = dataset[['sbjt_id', 'main_str', 'label']].loc[:100]

In [7]:
# dataset.to_csv('out_sample.txt', index=False, sep='\t')

In [ ]:
dataset = nlp.data.TSVDataset('out_sample.txt', field_indices=[1, 2], num_discard_samples=1)
# dataset_ = pd.read_csv('out_sample.txt', sep='\t')
# dataset_ = dataset_.drop(['label'], axis=1)

# field_indices=[1, 2] : 데이터가 1번 index, 레이블이 2번 index
# num_discard_samples=1 : 상단 1개의 row를 제외

In [58]:
# documents = []
# for doc in dataset_['main_str']:
#     documents.append([doc])

# ds = mxnet.gluon.data.SimpleDataset(documents)
# # ds = mxnet.gluon.data.SimpleDataset([['나 보기가 역겨워', '김소월']])
# tok = tokenizer.tokenize
# trans = nlp.data.BERTSentenceTransform(tok, max_seq_length=10, vocab=vocab)

In [59]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
    
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
    
    def __getitem__(self, i):
        return (self.sentences[i])

    def __len__(self):
        return (len(self.sentences))

In [60]:
data = BERTDataset(dataset, 1, tok, max_len, True, False)

AttributeError: 'function' object has no attribute 'vocab'

In [ ]:
for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
    optimizer.zero_grad()
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length

In [ ]:
def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
        attention_mask[i][:v] = 1
    return attention_mask.float()

# def forward(self, token_ids, valid_length, segment_ids):
attention_mask = gen_attention_mask(token_ids, valid_length)
    
_, pooler = bertmodel(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))

---

In [36]:
# ★
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [37]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [39]:
# ★
tok = tokenizer.tokenize

data_train = BERTDataset(dataset, 0, tok, vocab, max_len, True, False)

In [47]:
print(dataset)

In [45]:
def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
    if self.dr_rate:
        out = self.dropout(pooler)
    return self.classifier(out)

---

## KR-SBERT

In [63]:
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
model = SentenceTransformer('KR-SBERT/KR-SBERT-V40K-klueNLI-augSTS')

In [40]:
dataset = pq.read_table('tipa_text_tokens_20220328.parquet').to_pandas()
dataset = dataset.loc[:100]

In [57]:
documents = []
for doc in dataset['main_str']:
    documents.append(doc)


In [59]:
sentences = documents
vectors = model.encode(sentences)
similarities = cosine_similarity(vectors)
similarities

array([[0.9999999 , 0.49660236, 0.52995765, ..., 0.4263104 , 0.48406532,
        0.09445414],
       [0.49660236, 0.99999976, 0.4284887 , ..., 0.31369466, 0.43486148,
        0.1000298 ],
       [0.52995765, 0.4284887 , 0.99999994, ..., 0.41751313, 0.5828234 ,
        0.13223591],
       ...,
       [0.4263104 , 0.31369466, 0.41751313, ..., 1.0000005 , 0.49146536,
        0.16618317],
       [0.48406532, 0.43486148, 0.5828234 , ..., 0.49146536, 0.99999994,
        0.23146369],
       [0.09445414, 0.1000298 , 0.13223591, ..., 0.16618317, 0.23146369,
        0.9999998 ]], dtype=float32)

In [60]:
similarities.shape

(101, 101)

---

In [61]:
embedder = SentenceTransformer('KR-SBERT/KR-SBERT-V40K-klueNLI-augSTS')

corpus = documents

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [64]:
queries = ['인공지능',
           '인공지능과 자동차',
           '밸브의 고장 유무를 판단하는 인공지능']

top_k = 5
for query in queries:
    query_embedding = model.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx in top_results[0:top_k]:
        print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))





Query: 인공지능

Top 5 most similar sentences in corpus:
제품 개발 (Score: 0.2793)
유체점도 측정장비 개발 (Score: 0.2252)
차원 입체형상 당사에서 활용하고자 하는 Rapid Prototyping System이하 RP이란 쾌속조형기또는 rapid prototyping system이라고 하는데 설계 단계에 있는 차원 모델을 실용적이고 현실적인 모형이나 시제품Prototype으로 다른 중간 과정 없이 빠르게 생성하는 새로운 기술을 말합니다 CAD는 차원 데이터를 보유하고 있지만 그 표시는 모니터 화면 또는 그 Hard Copy이므로 출력할 때는 차원으로 밖에 되지 않습니다 반면 RP는 설계된 차원 CAD데이터를 차원 단면 데이터로 변환한 후 여러 가지 방법RP공정을 적용하여 순차적으로 적층해 감으로써 CAD데이터와 같은 차원 입체 형상을 제작하여 원하는 시제품을 얻어 낼수 있습니다 CAD상에서 설계한 데이터를 이용하여 바로 실물MOCKUP로 제작하고 만들어진 실물로부터 설계오류를 확인하여 궁극적으로 설계 완성도를 향상시키고 상품개발기간을 줄이고자 하는 목적으로 많이 활용되고 있다RP공정을 통해 얻은 모형은 당사의 주 원재료가 됩니다 오늘날 개방화 국제화 시대를 맞이하여 고객들의 의식수준이 향상되면서 다양한 제품에 대한 감각도 날로 향상되고 개성있는 제품을 선호하는 경향이 점점더 짙어지고 있습니다 따라서 각 기업들은 유수한 세계 기업들과 경쟁하기 위해서 원가절감 및 품질 혁신이 절실이 요구되고 다양한 고객들의 요구에 부합하기 위해서 다양한 제품을 생산해야 하는 실정입니다 또한 고객의 디자인에 대한 욕구를 만족 시키기 위해서 개성이 중시된 디자인 요소가 가미된 제품을 생산해야 됩니다또한 국내외 시장에서 경쟁이 치열해지면서 제품의 개발기간과 시작기간의 단축이 요청되고 있으며 이에 따른 시장 도입시기를 앞당기려는 노력이 경주되고 있으며 제품개발에 관련된 납기 단축이 중요한 문제로 대두되었습니다 특히 당사

---